In [6]:
pip install dash 

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install dash_bootstrap_components

Note: you may need to restart the kernel to use updated packages.


In [9]:
from dash import Dash, dcc, Output, Input, State, html
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
import plotly.express as px
import base64
import io

In [10]:
app = Dash(__name__, external_stylesheets=[dbc.themes.LUX])
titulo = dcc.Markdown(children='')
titulo_c = dcc.Markdown(children='# Cidade 1')
titulo_s = dcc.Markdown(children='# Cidade 2')

cities_list = []
options_types = []
options_year = ['Todos']
options_month = ['Todos']

grafico_city_1_per_day = dcc.Graph(figure={})
grafico_city_2_per_day = dcc.Graph(figure={})

grafico_city_1_per_hour = dcc.Graph(figure={})
grafico_city_2_per_hour = dcc.Graph(figure={})

grafico_city_1_heat = dcc.Graph(figure={})
grafico_city_2_heat = dcc.Graph(figure={})

grafico_city_2_h = dcc.Graph(figure={})
grafico_city_1_h = dcc.Graph(figure={})
# grafico_city_2_cluster = dcc.Graph(figure={})
# grafico_city_1_cluster = dcc.Graph(figure={})

dropdown_tipo = dcc.Dropdown(options=options_types)

botao = dbc.Button('Carregar', id='carregar_graficos')

dropdown_ano = dcc.Dropdown(options= options_year)
dropdown_mes = dcc.Dropdown(options=options_month)


app.layout = dbc.Container([
    html.Div(id='output-data-upload', style={
            'width':'30%',
            'fontWeight':'bold',
            'textAlign': 'center',
            'backgroundColor': 'yellow',
            'font-size':'16px',
            'borderRadius': '5px', 
            "alignSelf": "center"
        
    }),
    dbc.Col([
        dbc.Row([
            dbc.Col([titulo], style={
            'textAlign': 'center'
        })
        ]), 
    ]),
    
    dbc.Row([
       dcc.Upload(id='upload-data',
                  children="Adicione os arquivos aqui",
        style={
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
                  multiple=True
        )
    ]),
        dbc.Row([
        dbc.Col([dropdown_tipo]),
        dbc.Col([dropdown_ano]),
        dbc.Col([dropdown_mes]),
        dbc.Button('carregar', id='carregar', disabled=True)
    ], style={
            'textAlign': 'center'
        }),
    
    dbc.Row([
        dbc.Col([
            dbc.Row([titulo_c], justify='center'),
            dbc.Row([grafico_city_1_per_day]),
            dbc.Row([grafico_city_1_per_hour]),
            dbc.Row([grafico_city_1_heat]),
            dbc.Row([grafico_city_1_h]),
            
        ], style={
            'textAlign': 'center'
        }),
        dbc.Col([
            dbc.Row([titulo_s], justify='center'),
            dbc.Row([grafico_city_2_per_day]),
            dbc.Row([grafico_city_2_per_hour]),
            dbc.Row([grafico_city_2_heat]),
            dbc.Row([grafico_city_2_h]),
        ], style={
            'textAlign': 'center'
        }),
    ]),
    

#     dbc.Row([
#         dbc.Col([dropdown_ano]),
#     ]),
    
])

def parse_contents(contents, filename):
    content_type, content_string = contents.split(',')

    decoded = base64.b64decode(content_string)
    try:
        if 'csv' in filename:
            df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
    except Exception as e:
        print(e)
        return None
    
#     print(filename)
    return df
    
@app.callback(Output('output-data-upload', 'children'),
              Output('carregar', 'disabled'),
              Output(dropdown_tipo, 'options'),
              Output(dropdown_ano, 'options'),
              Output(dropdown_mes, 'options'),
              
              Input('upload-data', 'contents'), 
              
              State('upload-data', 'filename'),
              State(dropdown_tipo, 'options'),
              State(dropdown_ano, 'options'), 
              State(dropdown_mes, 'options'),
              
              prevent_initial_call=True)

def update_output(list_of_contents, list_of_names, options_type_crime, options_year, options_month):
    carregar_off = True
   # print(list_of_names)
    if list_of_contents is not None:
         if len(list_of_contents) == 2:
            cities_list.append(parse_contents(list_of_contents[0], list_of_contents[0]))
            cities_list.append(parse_contents(list_of_contents[1], list_of_names[1]))
            carregar_off = False
    else:
      #      print(list_of_contents)
            return html.Div(['There was an error processing this files.']), carregar_off, options_type_crime, options_year, options_month
    
    set_crimes_type = set(cities_list[0]['Category'].tolist()).intersection(set(cities_list[1]['Category'].tolist()))
    set_year = set(cities_list[0]['Year'].tolist()).intersection(set(cities_list[1]['Year'].tolist()))
    set_month = set(cities_list[0]['Month'].tolist()).intersection(set(cities_list[1]['Month'].tolist()))
    
    return html.Div(['ok']), carregar_off, list(set_crimes_type), options_year + list(set_year), options_month + list(set_month)

@app.callback(
    Output(grafico_city_1_per_day, 'figure'),
    Output(grafico_city_2_per_day, 'figure'),
    Output(grafico_city_1_per_hour, 'figure'),
    Output(grafico_city_2_per_hour, 'figure'),
    Output(grafico_city_1_heat, 'figure'),
    Output(grafico_city_2_heat, 'figure'),
    Output(grafico_city_1_h, 'figure'),
    Output(grafico_city_2_h, 'figure'),
    Output(titulo, 'children'),
    State(dropdown_tipo, 'value'),
    State(dropdown_ano, 'value'),
    State(dropdown_ano, 'options'),
    State(dropdown_mes, 'value'),
    State(dropdown_mes, 'options'),
    Input('carregar', 'n_clicks'),
    prevent_initial_call=True,
    )


def update_graph(crime_name, select_ano, options_ano, select_mes, options_mes, carregar):
    
    if isinstance(select_ano, str):
        ano = options_ano[1:]
    else:
        ano = [select_ano]
        
    if isinstance(select_mes, str):
        mes = options_mes[1:]
    else:
        mes = [select_mes]
        
    df1 = cities_list[0]
    df2 = cities_list[1]
    
    df_definitivo1 = df1.loc[df1['Category'].isin([crime_name])]
    df_definitivo1 = df_definitivo1.loc[df_definitivo1['Year'].isin([item for item in ano])]
    df_definitivo1 = df_definitivo1.loc[df_definitivo1['Month'].isin([item for item in mes])]
    
    df_definitivo2 = df2.loc[df2['Category'].isin([crime_name])]
    df_definitivo2 = df_definitivo2.loc[df_definitivo2['Year'].isin([item for item in ano])]
    df_definitivo2 = df_definitivo2.loc[df_definitivo2['Month'].isin([item for item in mes])]
    
    count_days_1 = pd.DataFrame(
    df_definitivo1['Day'].value_counts()).sort_index().reset_index().rename(columns={'Day':'Dia', 'count':'Quant'}
    )
    count_days_1['Crimes(%)'] = [((count_days_1['Quant'].loc[i] / count_days_1['Quant'].sum() * 100)) for i in range(count_days_1.shape[0])]
    
    count_days_2 = pd.DataFrame(
    df_definitivo2['Day'].value_counts()).sort_index().reset_index().rename(columns={'Day':'Dia', 'count':'Quant'}
    )
    count_days_2['Crimes(%)'] = [((count_days_2['Quant'].loc[i] / count_days_2['Quant'].sum() * 100)) for i in range(count_days_2.shape[0])]
    
#     print(count_days_1)
    fig_city_1_crimes_per_day = px.bar(count_days_1,
                  x='Dia',
                  y='Crimes(%)',
                  color = 'Crimes(%)',
                  color_continuous_scale='pinkyl'
                        )
    fig_city_2_crimes_per_day = px.bar(count_days_2,
                  x='Dia',
                  y='Crimes(%)',
                  color = 'Crimes(%)',
                  color_continuous_scale='pinkyl'
                 )
    
    count_time_1 = pd.DataFrame(
        df_definitivo1['Time'].value_counts()).sort_index().reset_index().rename(columns={'count':'Quant', 'Time':'Hora'}
    )
    count_time_2 = pd.DataFrame(
        df_definitivo2['Time'].value_counts()).sort_index().reset_index().rename(columns={'count':'Quant', 'Time':'Hora'}
    )
    
    fig_city_1_crimes_per_hour = px.bar(count_time_1,
                    x='Hora',
                    y = 'Quant',
                    color = 'Quant',
                    color_continuous_scale='pinkyl'
                 )
    fig_city_2_crimes_per_hour = px.bar(count_time_2,
                    x='Hora',
                    y = 'Quant',
                    color = 'Quant',
                    color_continuous_scale='pinkyl'
                 )
    
    fig_city_1_heatmap = px.density_mapbox(df_definitivo1, lat='Latitude', lon='Longitude', radius=7, center=dict(lat=df_definitivo1.Latitude.mean(), 
                                lon=df_definitivo1.Longitude.mean()), zoom=10.5,
                                mapbox_style="open-street-map", height=900, opacity=0.8, color_continuous_scale='pinkyl')
    fig_city_2_heatmap = px.density_mapbox(df_definitivo2, lat='Latitude', lon='Longitude', radius=7, center=dict(lat=df_definitivo2.Latitude.mean(), 
                                lon=df_definitivo2.Longitude.mean()), zoom=11.5,
                                mapbox_style="open-street-map", height=900, opacity=0.8, color_continuous_scale='pinkyl')

    pt_1 = pd.pivot_table(df_definitivo1,index="Time",columns="DayOfWeek",aggfunc=len,fill_value=0)["Day"]
    print(df_definitivo1.DayOfWeek.unique())
    pt_1 = pt_1[['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']]
    pt_1.columns = ['Domingo','Segunda','Terça','Quarta','Quinta','Sexta','Sábado']
    heat_1 = px.imshow(pt_1, color_continuous_scale='pinkyl')
    
    pt_2 = pd.pivot_table(df_definitivo2,index="Time",columns="DayOfWeek",aggfunc=len,fill_value=0)["Day"]
    pt_2 = pt_2[['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']]
    pt_2.columns = ['Domingo','Segunda','Terça','Quarta','Quinta','Sexta','Sábado']
    heat_2 = px.imshow(pt_2, color_continuous_scale='pinkyl')
  
    return fig_city_1_crimes_per_day, fig_city_2_crimes_per_day, fig_city_1_crimes_per_hour, fig_city_2_crimes_per_hour, fig_city_1_heatmap, fig_city_2_heatmap, heat_1, heat_2, '# Crimes'

if __name__ == '__main__':
#     app.run_server(port=8054, debug=True, use_reloader=False)
    app.run(jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/


[2023-11-16 14:52:08,418] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Geniar\anaconda3\Lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Geniar\anaconda3\Lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Geniar\anaconda3\Lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Geniar\anaconda3\Lib\site-packages\flask\app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Geniar\anaconda3\Lib\site-packages\dash\dash.py", line 1310, in dispatch
    ctx.run

['Thursday' 'Wednesday' 'Sunday' 'Saturday' 'Friday']
['Thursday' 'Wednesday' 'Tuesday' 'Friday' 'Monday' 'Saturday' 'Sunday']
['Thursday' 'Sunday' 'Friday' 'Tuesday' 'Wednesday' 'Monday' 'Saturday']
